In [1]:
import pandas as pd 
import numpy as np
import pickle

DATA_PATH = './data/'

In [2]:
with open(f"{DATA_PATH}train.pkl", 'rb') as f:
    data = pickle.load(f)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
train, valid = train_test_split(data, test_size=0.2, random_state=42, stratify=data['mbti'], shuffle=True)



y_train = train['mbti']
X_train = train.drop('mbti', axis=1)
y_valid = valid['mbti']
X_valid = valid.drop('mbti', axis=1)

# Convert y_train and y_valid to separate binary columns for each MBTI character
y_train_binary = pd.DataFrame({
    'I-E': y_train.apply(lambda x: 1 if x[0] == 'I' else 0),
    'N-S': y_train.apply(lambda x: 1 if x[1] == 'N' else 0),
    'T-F': y_train.apply(lambda x: 1 if x[2] == 'T' else 0),
    'J-P': y_train.apply(lambda x: 1 if x[3] == 'J' else 0)
})

y_valid_binary = pd.DataFrame({
    'I-E': y_valid.apply(lambda x: 1 if x[0] == 'I' else 0),
    'N-S': y_valid.apply(lambda x: 1 if x[1] == 'N' else 0),
    'T-F': y_valid.apply(lambda x: 1 if x[2] == 'T' else 0),
    'J-P': y_valid.apply(lambda x: 1 if x[3] == 'J' else 0)
})

In [4]:
from catboost import CatBoostClassifier

# List to store models and performance metrics
models = {}
accuracy_scores = {}
f1_scores = {}

# Train and evaluate a model for each binary classification problem
for column in y_train_binary.columns:
    # Initialize the CatBoostClassifier
    model = CatBoostClassifier(verbose=0, task_type="GPU", devices='0:1')
    
    # Fit the model
    model.fit(X_train, y_train_binary[column])
    
    # Predict on the validation set
    y_pred = model.predict(X_valid)
    
    # Calculate accuracy and F1 score
    accuracy = accuracy_score(y_valid_binary[column], y_pred)
    f1 = f1_score(y_valid_binary[column], y_pred)
    
    # Store the model and metrics
    models[column] = model
    accuracy_scores[column] = accuracy
    f1_scores[column] = f1
    
    # Print the results
    print(f"Results for {column}:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print("-" * 30)

Results for I-E:
Accuracy: 0.7889063108629414
F1 Score: 0.8819470553110555
------------------------------
Results for N-S:
Accuracy: 0.926119186075251
F1 Score: 0.9616365135378496
------------------------------
Results for T-F:
Accuracy: 0.6248657925800845
F1 Score: 0.6794553598129341
------------------------------
Results for J-P:
Accuracy: 0.6116971529242883
F1 Score: 0.259929076677834
------------------------------


In [5]:
from xgboost import XGBClassifier

# List to store models and performance metrics
models = {}
accuracy_scores = {}
f1_scores = {}

# Train and evaluate a model for each binary classification problem
for column in y_train_binary.columns:
    # Initialize the CatBoostClassifier
    model = XGBClassifier(device="cuda")
    
    # Fit the model
    model.fit(X_train, y_train_binary[column])
    
    # Predict on the validation set
    y_pred = model.predict(X_valid)
    
    # Calculate accuracy and F1 score
    accuracy = accuracy_score(y_valid_binary[column], y_pred)
    f1 = f1_score(y_valid_binary[column], y_pred)
    
    # Store the model and metrics
    models[column] = model
    accuracy_scores[column] = accuracy
    f1_scores[column] = f1
    
    # Print the results
    print(f"Results for {column}:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print("-" * 30)

/mnt/c/Users/zhang/Code/MBTI_Classifier/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [09:36:01] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Results for I-E:
Accuracy: 0.789062515447447
F1 Score: 0.8819199015344292
------------------------------
Results for N-S:
Accuracy: 0.9262971406651943
F1 Score: 0.9617189353396617
------------------------------
Results for T-F:
Accuracy: 0.6283794070948517
F1 Score: 0.6794415222033471
------------------------------
Results for J-P:
Accuracy: 0.6161875404105215
F1 Score: 0.32479024919647703
------------------------------


In [7]:
from lightgbm import LGBMClassifier 


# List to store models and performance metrics
models = {}
accuracy_scores = {}
f1_scores = {}

# Train and evaluate a model for each binary classification problem
for column in y_train_binary.columns:
    # Initialize the CatBoostClassifier
    model = LGBMClassifier(verbose=0)
        
    # Fit the model
    model.fit(X_train, y_train_binary[column])
    
    # Predict on the validation set
    y_pred = model.predict(X_valid)
    
    # Calculate accuracy and F1 score
    accuracy = accuracy_score(y_valid_binary[column], y_pred)
    f1 = f1_score(y_valid_binary[column], y_pred)
    
    # Store the model and metrics
    models[column] = model
    accuracy_scores[column] = accuracy
    f1_scores[column] = f1
    
    # Print the results
    print(f"Results for {column}:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print("-" * 30)

Results for I-E:
Accuracy: 0.7889774926989186
F1 Score: 0.8819774668571707
------------------------------
Results for N-S:
Accuracy: 0.9262081633702227
F1 Score: 0.9616792143622854
------------------------------
Results for T-F:
Accuracy: 0.6233670194781185
F1 Score: 0.6780931071254143
------------------------------
Results for J-P:
Accuracy: 0.6115923574435439
F1 Score: 0.26187398545061025
------------------------------
